In [69]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import binom
from scipy import interpolate, integrate
from itertools import combinations

In [70]:
def randpoints(n, scale=1):
    xp = np.random.random(n) * scale
    xp = np.append(xp,xp[0]) # need duplicate points for periodic boundary
    return xp

def randthetas(n): # Not used at the moment
    """ Ensures that we cover the whole range."""
    thetas = randpoints(n, scale=2*np.pi)
    thetas[0] = 0
    thetas[-1] = 2*np.pi
    return thetas


def halve(ls):
    """ Splits a list of two sublists in half"""
    ls1 = ls[0][len(ls[0])//2:] ,(ls[1][len(ls[1])//2:])
    ls2 = ls[0][:len(ls[0])//2] ,(ls[1][:len(ls[1])//2])
    return ls1,ls2

def breakup(ls,order):
    """Breaks up ls into 2^order sublists"""
    if order == 1: # base case
        return halve(ls)
    else:
        ls1, ls2 = halve(ls)
        return breakup(ls1,order-1) + breakup(ls2,order-1)

def range_intersects(rng1,rng2):
    """ rng = (xmin, xmax)"""
    return (rng1[0] <= rng2[0] <= rng1[1] or 
            rng2[0] <= rng1[0] <= rng2[1] or
            rng1[0] <= rng2[1] <= rng1[1] or
            rng2[0] <= rng1[1] <= rng2[1])    
    
def box_intersects(box1,box2):
    """ box = (xmin,xmax,ymin,ymax)"""
    return (range_intersects((box1[0],box1[1]),(box2[0],box2[1])) and
            range_intersects((box1[2],box1[3]),(box2[2],box2[3]))  )

def curve_intersects_rec(c1,c2):
    if len(c1[0]) <= THRESH: # base case - should small enough to say there's an intersection there
        return True # NOTE - the above value is arbitrary
    # construct bounding boxes
    box1 = (np.min(c1[0]),np.max(c1[0]),np.min(c1[1]),np.max(c1[1]))
    box2 = (np.min(c2[0]),np.max(c2[0]),np.min(c2[1]),np.max(c2[1]))
    
    if SHOW_STEPS:
        plt.figure()
        plt.axis((-1,2,-1,2))
        plt.plot(c1[0],c1[1],c2[0],c2[1])
        plt.title(str(box_intersects(box1,box2)) + ' size =' + str(len(c1[0])))
        plt.show()
    
    if box_intersects(box1,box2): #split the curves in half, recurse
        c1a, c1b = halve(c1)
        c2a, c2b = halve(c2)
        return (curve_intersects_rec(c1a,c2a) or 
                curve_intersects_rec(c1a,c2b) or
                curve_intersects_rec(c1b,c2a) or
                curve_intersects_rec(c1b,c2b) )
    else:
        return False

def curve_intersects(c):
    """ Takes as input two curves c1 = [x,y]
    Returns True if c1 and c2 intersect. 
    Works by recursing on bounding boxes.
    Thanks to the lovely Pomax for the method."""
    assert len(c[0]) == len(c[1])
    assert len(c[0]) > 10 # it'll give true by default if you start with a small list

    # VERY Hacky fix - some self-intersections get lost if you don't break it up enough
    # This is where the few bad apples slip through. 
    cs = breakup(c,3)
    c_pairs = combinations(cs,r=2) # try each combination

    if SHOW_FAILS:
        plt.figure()
        plt.axis((-0.5,1.5,-0.5,1.5))
        for curve in cs:
            plt.plot(curve[0],curve[1])
        plt.show()
    
    
    for pair in c_pairs:
        if curve_intersects_rec(pair[0],pair[1]):
            return True
       
    if SHOW_WINS:
        plt.figure()
        plt.axis((-0.5,1.5,-0.5,1.5))
        for curve in cs:
            plt.plot(curve[0],curve[1])
        plt.show()
    
    return False

In [74]:
RAND_POINTS = 5
THRESH = 100
C_POINTS = 2000
SHOW_STEPS = False
SHOW_WINS = True
SHOW_FAILS = False
VIEWS = 1

In [77]:
if __name__ == "__main__":
    views = 0
    while views < VIEWS:
        valid_shape = False
        failcounter = 0
        while not valid_shape:
            x,y = randpoints(RAND_POINTS), randpoints(RAND_POINTS)
            tck, u = interpolate.splprep([x, y], s=0, per=True)
            unew = np.linspace(0,1,C_POINTS) # NOTE - can tweak the resolution if needed
            xnew,ynew = interpolate.splev(unew, tck)
            xnew = np.delete(xnew,0) # need to remove duplicate points
            ynew = np.delete(ynew,0)
            if curve_intersects((xnew,ynew)):
                failcounter += 1
                continue
            valid_shape = True
            print str(failcounter) + ' fails'
#             plt.figure()
#             plt.plot(x, y, 'x', xnew, ynew, 'b')
#             plt.legend(['points', 'Cubic Spline'])
#             plt.title('Potato')
#             plt.show()
            views+=1

20 fails
